# Discriminative Sparse Coding

### import libraries

In [1]:
from __future__ import division
import numpy as np
import pandas as pd
import time
import librosa
import pickle
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import SparseCoder,DictionaryLearning
from sklearn import cluster
from sklearn.preprocessing import normalize 
# from lightning.regression import CDRegressor
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import warnings
import psutil
warnings.filterwarnings("ignore")

In [2]:
class DDSC():
    def __init__(self, train_set, train_sum, alpha, 
                 epsilon, reg_lambda, steps, n, m, T, k):
        """
        Inputs:
            train_set: dict of X_i matrix with dim T*m for each individual appliance i 
            train_sum: dataframe of X_sum aggregated matrix T*m 
            alpha: gradiant rate for the convergence step for DD (4b).
            epsilon: gradient stepsize of the pre-training (2e) ||A_t+1 - A_t||< epsilon 
            reg_lambda: reguarization weight of penalty function
            steps: interations to be performed for the convergence part
            n: number of basis functions 
            m: number of features (households)
            T: number of samples (hours)
            k: number of applicances i (1, k)
        """
        self.train_set = train_set.values()
        self.train_sum = train_sum.values
        self.alpha = alpha 
        self.epsilon = epsilon
        self.reg_lambda = reg_lambda
        self.steps = steps
        self.n = n 
        self.m = m
        self.T = T
        self.k = k
        
        # ======= Instances that can be used for plotting =====
        self.acc_nnsc = None
        self.err_nnsc = None
        self.acc_ddsc = None
        self.err_ddsc = None
        
        self.a_nnsc = None
        self.b_nnsc = None
        self.a_ddsc = None
        self.b_ddsc = None

    def _initialization(self):
        '''
        DDSC step 1
        initiualize the matrices A,B with positive values
        scale columns of B s.t b(j) = 1
        '''
        A = np.random.random((self.n,self.m)) # A: n*m
        B = np.random.random((self.T,self.n)) # B: T*n

        # scale columns s.t. b_i^(j) = 1
        B /= sum(B) 
        
        return A, B
    
    @staticmethod
    def _pos_constraint(mat):
        '''
        nnsc step 2(b)
        using only the positive values of matrix  
        input: matrix n*m 
        '''     
        indices = np.where(mat < 0.0)
        mat[indices] = 0.0
        return mat   
    
    def nnsc(self):
        '''
        Method as in NNSC from nonnegative sparse coding finland.
        from P.Hoyer

        return:
            A_list, B_list: list of A and B for each appliance i 
        '''
        
        acc_nnsc = []
        err_nnsc = []
        a_nnsc = []
        b_nnsc = []
        
        # used for F
        X_train = self.train_set # dict_value 
        A_list = []
        B_list = []
        
        for X in X_train:
            # step 1 
            A0, B0 = self._initialization() # initialization 
            Ap, Bp = A0, B0 
            Ap1, Bp1 = Ap, Bp # record previous step Ap, Bp
            t = 0
            change_A = 1.0
            while t <= self.steps and change_A >= self.epsilon:            
                Bp = Bp - self.alpha * np.dot((np.dot(Bp, Ap) - X), Ap.T) # step 2a
                Bp = self._pos_constraint(Bp) # step 2b 
#                 Bp /= sum(Bp) # step 2c 
                Bp = normalize(Bp, norm='l2', axis=0)
                
                # step 2d
                dot_part2 = np.divide(np.dot(Bp.T, X), (np.dot(np.dot(Bp.T, Bp), Ap) + self.reg_lambda)) # element wise division 
                Ap = np.multiply(Ap, dot_part2)

                change_A = np.linalg.norm(Ap - Ap1)
                change_B = np.linalg.norm(Bp - Bp1)
                Ap1, Bp1 = Ap, Bp
                t += 1
                
                if t % 10 == 0:
                    print("iter {t}：A change = {a:8.4f}".format(t=t, a=change_A))
                
            print("Gone through one appliance.\n")
            A_list.append(Ap)
            B_list.append(Bp)


        # for thesis
        acc_iter = self.accuracy(X_train, self.train_sum, B_list, A_list)
        err_iter = self.error(X_train, self.train_sum, B_list, A_list)
        acc_nnsc.append(acc_iter)
        err_nnsc.append(err_iter)
        # append norm of matrices
        a_nnsc.append(np.linalg.norm(sum(A_list)))
        b_nnsc.append(np.linalg.norm(sum(B_list)))

        self.acc_nnsc = acc_nnsc
        self.err_nnsc = err_nnsc
        self.a_nnsc = a_nnsc
        self.b_nnsc = b_nnsc
        
        return A_list, B_list

    def accuracy(self, X_train, X_sum, B, A):
        '''
        inputs:
            X_train: dict_value of list 
        
        Everything needs to be in lists of ndarrays
        of the components
        '''
        B_cat = np.hstack(B)
        A_cat = np.vstack(A)

        A_prime = self.F(X_sum, B_cat, A=A_cat)
        A_last = np.split(A_prime, self.k, axis=0)
        X_predict = self.predict(A_last, B)
        
        
        X_train = list(X_train)
        

        acc_numerator = [np.sum(np.minimum((B[i].dot(A_last[i])).sum(axis=0), (sum(X_train[i].sum(axis=0)))))
                         for i in range(len(B))]
        
        
        acc_denominator = sum(X_predict).sum()
        acc = sum(acc_numerator) / acc_denominator
        
        acc_denominator = X_sum.sum()
        acc_star = sum(acc_numerator) / acc_denominator
        return acc, acc_star

    def get_accuracy_plot(self):
        return self.acc_nnsc, self.acc_ddsc

    def get_error_plot(self):
        return self.err_nnsc, self.err_ddsc

    def get_a(self):
        return self.a_nnsc, self.a_ddsc

    def get_b(self):
        return self.b_nnsc, self.b_ddsc

    def error(self,X, X_sum, B, A):
        '''
        Error for the whole disaggregation part within list, sum the list to get
        the resulting disaggregation
        Parameters : must have x_train as x
        '''
        B_cat = np.hstack(B)
        A_cat = np.vstack(A)
        
        
        error = [(1.0/2.0) * np.linalg.norm((list(X)[i] - B[i].dot(A[i]))**2) for i in range(self.k)]
        error = sum(error)
        
        A_last_error = self.F(X_sum, B_cat,A_cat)
        
        A_last_error_list = np.split(A_last_error,self.k,axis=0)
        error_star = [(1.0/2.0) * np.linalg.norm((list(X)[i] - B[i].dot(A_last_error_list[i]))**2) for i in range(self.k)]
        error_star = sum(error_star)
        return error, error_star
        
    
    def F(self, X_sum, B, A):
        '''
        input is lists of the elements
        output list of elements
        '''
        # 4a  
        B = np.asarray(B)
        A = np.asarray(A)
        
        coder = SparseCoder(dictionary=B.T, transform_alpha=self.reg_lambda, transform_algorithm='lasso_cd')    
        # B: basis function 
        # A: activation function   
        B_hat, A_hat = librosa.decompose.decompose(X_sum, transformer=coder) 
        A_hat = self._pos_constraint(A_hat)

        return A_hat

    def DD(self, B, A):
        '''
        Taking the parameters as x_train_use and discriminate over the
        entire region
        '''
        # step 3
        A_star = np.vstack(A)
        B_cat = np.hstack(B)
        
        # step 4 
        change_B = 1 
        t = 0
        
        acc_ddsc = []
        err_ddsc = []
        a_ddsc = []
        b_ddsc = []
        
        X_sum = self.train_sum # change df to list of list   
        X_train = self.train_set
        
        while t <= self.steps and self.epsilon <= change_B:
            B_cat_p = B_cat
            
            # step 4a
            A_hat = self.F(X_sum, B_cat, A_star)
            
            # step 4b
            B_cat = (B_cat - self.alpha * ((X_sum - B_cat.dot(A_hat)).dot(A_hat.T) - (X_sum - B_cat.dot(A_star)).dot(A_star.T)))
            
            # step 4c
            B_cat = self._pos_constraint(B_cat) # scale columns s.t. b_i^(j) = 1
#             B_cat /= sum(B_cat)
            
            B_cat = normalize(B_cat, norm='l2', axis=0)
            
            change_B = np.linalg.norm(B_cat - B_cat_p)
            t += 1
            
    
            print("step {t}: B change = {c:.4f}".format(t=t, c=change_B))

            # convergence check
            A_hat_split = np.split(A_hat, self.k, axis=0)
            B_split = np.split(B_cat,self.k,axis=1)
            
            acc_iter = self.accuracy(X_train, X_sum, B, A_hat_split)
            acc_iter = self.accuracy(X_train, X_sum, B_split, A)
            err_iter = self.error(X_train, X_sum, B, A_hat_split)

#             error, error_star = sc.error(list(x_train.values()),train_sum,B_list,A_list)

               
            acc_ddsc.append(acc_iter)
            err_ddsc.append(err_iter)
            a_ddsc.append(np.linalg.norm(A_hat))
            b_ddsc.append(np.linalg.norm(B_cat))

        self.acc_ddsc = acc_ddsc
        self.err_ddsc = err_ddsc
        self.a_ddsc = a_ddsc
        self.b_ddsc = b_ddsc
        return B_cat

    def predict(self, A, B):
        result = [x.dot(y) for (x, y) in zip(B, A)]
        return result 

In [3]:
from dataprocess import read_data, format_data, split, split2
df, houses = read_data()

d = format_data(df, houses)

timeframe = 336
portion = 0.5

# x_train, x_test = split(d, portion, timeframe)

start_t = 2000
x_train, x_test = split2(d, portion, timeframe, start_t)
x_train_sum = x_train.pop('use',None) # aggregated 
x_test_sum = x_test.pop('use',None) 
x_train_localhour = x_train.pop('localhour',None)
x_test_localhour = x_test.pop('localhour',None)

k = len(x_train.keys())
T, m = x_train[list(x_train.keys())[0]].shape
reg_par = 0.0005
epsilon = 0.001
alpha = 0.001
n = 150
steps = 200 # steps must be higher than k


sc = DDSC(x_train, x_train_sum, alpha, epsilon, reg_par, steps, n, m, T, k)


print('pre-training: ')
A_list,B_list = sc.nnsc()

# print('DD: ')
# # Discriminative Disaggregation training
# B_cat = sc.DD(B_list, A_list)


# # Given test examples x_test
# A_prime = sc.F(x_test_sum.values, B_cat, A=np.vstack(A_list))
# A_last = np.split(A_prime,k,axis=0)

x_predict = sc.predict(A_list,B_list)

range(2000, 2168)
range(2168, 2336)
pre-training: 
iter 10：A change =   0.2213
iter 20：A change =   0.2008
iter 30：A change =   0.2340
iter 40：A change =   0.2254
iter 50：A change =   0.1789
iter 60：A change =   0.1534
iter 70：A change =   0.1382
iter 80：A change =   0.1197
iter 90：A change =   0.0981
iter 100：A change =   0.0803
iter 110：A change =   0.0673
iter 120：A change =   0.0592
iter 130：A change =   0.0544
iter 140：A change =   0.0518
iter 150：A change =   0.0505
iter 160：A change =   0.0494
iter 170：A change =   0.0484
iter 180：A change =   0.0468
iter 190：A change =   0.0446
iter 200：A change =   0.0416
Gone through one appliance.

iter 10：A change =   0.1184
iter 20：A change =   0.0913
iter 30：A change =   0.1073
iter 40：A change =   0.1554
iter 50：A change =   0.2010
iter 60：A change =   0.1397
iter 70：A change =   0.0532
iter 80：A change =   0.0176
iter 90：A change =   0.0069
iter 100：A change =   0.0039
iter 110：A change =   0.0030
iter 120：A change =   0.0028
iter 130：A

In [4]:
print("the shape of the first predicted appliances is :%s" %(np.asarray(x_predict[1]).shape,))
x_predict_sum = sum(x_predict)

# energy disaggregation accuracy
acc = sc.accuracy(x_train.values(), x_train_sum, B_list, A_last)
# energy disaggregation error
error, error_star = sc.error(x_train.values(), x_train_sum, B_list, A_list)

print("error: %s, error_star: %s" % (error, error_star))
acc_nnsc, acc_ddsc = sc.get_accuracy_plot()
err_nnsc, err_ddsc = sc.get_error_plot()
# plotting acc/err
a_nnsc, a_ddsc = sc.get_a()
b_nnsc, b_ddsc = sc.get_b()

the shape of the first predicted appliances is :(168, 21)


NameError: name 'A_last' is not defined

In [ ]:
a_nnsc, a_ddsc = sc.get_a()
b_nnsc, b_ddsc = sc.get_b()
plt.plot(a_ddsc)
plt.ylabel('Activations')
plt.xlabel('Iterations')
plt.title('Activation Norm for DDSC algorithm')
plt.show()

plt.plot(b_ddsc)
plt.ylabel('Basis')
plt.xlabel('Iterations')
plt.title('Basis Norm for DDSC algorithm')
plt.show()

In [ ]:
print("the shape of the first predicted appliances is :%s" %(np.asarray(list(x_predict)[1]).shape,))


In [ ]:
err_nnddsc, err_ddddsc = sc.get_error_plot()
res_err = list(zip(*err_ddddsc)) 
plt.plot(res_err[0])
plt.title('error of DDSC algorithm')
plt.xlabel('Iterations')
plt.show()

acc_nnsc, acc_ddsc = sc.get_accuracy_plot()
res_acc = list(zip(*acc_ddsc)) 
plt.plot(res_acc[0])
plt.title('accuracy of DDSC algorithm')
plt.xlabel('Iterations')
plt.show()


In [ ]:
x_predict = sc.predict(A_last,B_list)
print("the shape of the first predicted appliances is :%s" %(np.asarray(list(x_predict)[0]).shape,))
x_test.keys()

In [5]:
x_predict = sc.predict(A_list,B_list)
# pred_val = np.asarray(list(x_predict)[4])[:,6]
# pred_val 

In [6]:
pred_val = np.asarray(list(x_predict)[2])[:,6]
pred_val

array([0.03503577, 0.03272738, 0.0321988 , 0.03533284, 0.06002436,
       0.05413528, 0.00251546, 0.05646066, 0.03048156, 0.03477347,
       0.04630066, 0.05412129, 0.08144623, 0.08874165, 0.04940897,
       0.05816919, 0.05239363, 0.05033131, 0.04757575, 0.0319756 ,
       0.02993014, 0.02073146, 0.02774004, 0.04241538, 0.03930864,
       0.03579884, 0.02295173, 0.05273446, 0.02449715, 0.07238389,
       0.07207574, 0.05326708, 0.02289917, 0.05241542, 0.03394401,
       0.05807873, 0.05323182, 0.03876631, 0.02434633, 0.0406014 ,
       0.02581101, 0.04308122, 0.04361633, 0.02755116, 0.03800951,
       0.02394727, 0.04205523, 0.02202511, 0.05712127, 0.05280344,
       0.04841218, 0.06817729, 0.11352683, 0.07292208, 0.05140763,
       0.03644648, 0.0179421 , 0.04560025, 0.05964163, 0.04197712,
       0.06299369, 0.04644025, 0.07518429, 0.01570311, 0.0352393 ,
       0.02448276, 0.01983692, 0.03838524, 0.05629954, 0.0388362 ,
       0.06655321, 0.02075051, 0.0491967 , 0.06103187, 0.05279

In [7]:
x = range(x_train[list(x_train.keys())[0]].shape[0])
y_other_true = np.asarray(x_test[list(x_test.keys())[4]])[0:,house]
y_other_pred = np.asarray(list(x_predict)[4])[0:,house]

result_df = pd.DataFrame({'actuals':y_other_true, 'predicted':y_other_pred})
result_df.head()

NameError: name 'house' is not defined

In [67]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

house = 10

y_use_true = np.asarray(x_test_sum)[0:,house]
x_predict = sc.predict(A_list,B_list)

def plot_predict(x_test, x_predict, house, x_test_localhour):
#     fig = make_subplots(rows=5, cols=1, subplot_titles=('air', 'furnace', 'dishwasher', 'regrigerator', 'other'))
    y_axis = ['air', 'furnace', 'dishwasher', 'refrigerator', 'other']
    fig = make_subplots(rows=5, cols=1)
    dates = x_test_localhour.iloc[:, house]
    
    for i in range(len(x_test)):
        if i == 0:
            flag = True
        else:
            flag = False
        
        true_val = np.asarray(x_test[list(x_test.keys())[i]])[:,house]
#         x_predict = sc.predict(A_list,B_list)
        pred_val = np.asarray(list(x_predict)[i])[:,house]
        true_pt = go.Scatter(name= 'Actuals(kWh)',
                     x= dates,
                     y= true_val,
                    xaxis='x2', yaxis='y2',
                     mode='lines',
                     marker=dict(size=12,
                                 line=dict(width=1),
                                 color='blue'), showlegend=flag)
        
        pred_pt = go.Scatter(name= 'Predicted(kWh)',
                     x= dates,
                     y= pred_val,
                    xaxis='x2', yaxis='y2',
                     mode='lines',
                     marker=dict(size=12,
                                 line=dict(width=1),
                                 color="red"), showlegend=flag)
        fig.append_trace(true_pt, row=i+1, col=1)
        fig.append_trace(pred_pt, row=i+1, col=1)
        fig.update_yaxes(title_text=y_axis[i], row=i+1, col=1)
    
    layout=go.Layout(title='A week of daily consumption for a household', 
                     xaxis={'title':'Hour'}, yaxis={'title':'kW'}, template="plotly_white")

    fig.update_layout(showlegend=True, height=1000, width=800, title_text="Test set prediction", 
                      template="plotly_white", font=dict(
        family="Arial",
        size=18
    ))
    fig.update_layout(legend=dict(x=0.72, y=1))
    fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True)
    fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True)
    fig.show()
    

    
plot_predict(x_test, x_predict, house, x_test_localhour)

In [58]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def pie_plot(x_test, x_predict, house):
    
    labels = list(x_test.keys())
    
    pie_chart_true = []
    pie_chart_pred = []
    for i in range(len(x_test)):
        true_val = np.asarray(x_test[list(x_test.keys())[i]])[:,house]
        pred_val = np.asarray(list(x_predict)[i])[:,house]
        pie_chart_true.append(true_val.sum())
        pie_chart_pred.append(pred_val.sum())
        
    # Create subplots: use 'domain' type for Pie subplot
    fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]], 
                        subplot_titles=['True usage', 'Predicted usage'])

    fig.add_trace(go.Pie(labels=labels, values=pie_chart_true, name="True usage"),
                  1, 1)
    fig.add_trace(go.Pie(labels=labels, values=pie_chart_pred, name="Predicted usage"),
                  1, 2)

    fig.update_traces(textposition='inside', textinfo='percent+label')
    fig.update_layout(title_text='percent usage comparsion', font=dict(
        family="Arial",
        size=20
    ))
    fig.update_layout(legend=dict(x=0.4, y=1.1))
    fig.show()

pie_plot(x_test, x_predict, house=3)

In [35]:
def generate_df_detect_anomalies(d, house, appliance, x_test, x_test_localhour, x_predict): 
    
    # {'air1': 0, 'furnace1': 1, 'dishwasher1': 2, 'regrigerator1': 3, 'other': 4}
    labels = dict(zip(list(x_test.keys()), range(len(x_test))))
    
    all_hours = d['localhour'].iloc[:, house]   
    all_true = d[appliance].iloc[:, house]
    true_df = pd.concat([all_hours, all_true], axis=1)   
    true_df.columns = ['date', 'actuals']
    
    test_hours = x_test_localhour.iloc[:, house]
    pred_val = pd.Series(x_predict[labels[appliance]][:,house])
    pred_val.index = test_hours.index 
    pred_df = pd.concat([test_hours, pred_val], axis=1)
    pred_df.columns = ['date', 'predicted']

    result_df = pd.merge(true_df, pred_df, on='date', how= 'outer')
    return result_df

result_df = generate_df_detect_anomalies(d, 6, 'other', x_test, x_test_localhour, x_predict)
result_df.head()

,date,actuals,predicted
0,2014-01-01 00:00:00,1.200,NaN
1,2014-01-01 01:00:00,0.473,NaN
2,2014-01-01 02:00:00,0.439,NaN
3,2014-01-01 03:00:00,0.388,NaN
4,2014-01-01 04:00:00,0.427,NaN


In [36]:
def detect_classify_anomalies(df,window):
#     df.replace([np.inf, -np.inf], np.NaN, inplace=True)
#     df.fillna(0,inplace=True)
    df['error']=df['actuals']-df['predicted']
    df['percentage_change'] = ((df['actuals'] - df['predicted']) / df['actuals']) * 100
    df['meanval'] = df['error'].rolling(window=window).mean()
    df['deviation'] = df['error'].rolling(window=window).std()
    df = df.dropna(how='any')
    df = df.reset_index(drop=True)
    df['-3s'] = df['meanval'] - (2 * df['deviation'])
    df['3s'] = df['meanval'] + (2 * df['deviation'])
    df['-2s'] = df['meanval'] - (1.75 * df['deviation'])
    df['2s'] = df['meanval'] + (1.75 * df['deviation'])
    df['-1s'] = df['meanval'] - (1.5 * df['deviation'])
    df['1s'] = df['meanval'] + (1.5 * df['deviation'])
    cut_list = df[['error', '-3s', '-2s', '-1s', 'meanval', '1s', '2s', '3s']]
    cut_values = cut_list.values
    cut_sort = np.sort(cut_values)
    df['impact'] = [(lambda x: np.where(cut_sort == df['error'][x])[1][0])(x) for x in
                               range(len(df['error']))]
    severity = {0: 3, 1: 2, 2: 1, 3: 0, 4: 0, 5: 1, 6: 2, 7: 3}
    region = {0: "NEGATIVE", 1: "NEGATIVE", 2: "NEGATIVE", 3: "NEGATIVE", 4: "POSITIVE", 5: "POSITIVE", 6: "POSITIVE",
              7: "POSITIVE"}
    df['color'] =  df['impact'].map(severity)
    df['region'] = df['impact'].map(region)
    df['anomaly_points'] = np.where(df['color'] == 3, df['error'], np.nan)
    df['load_date'] = pd.date_range(start='6/1/2014', periods=len(df), freq='H')
#     df = df.sort_values(by='load_date', ascending=False)
#     df.load_date = pd.to_datetime(df['load_date'].astype(str), format="%Y-%m-%d")
    return df

df = detect_classify_anomalies(result_df, window=12)
df.head(10)


,date,actuals,predicted,error,percentage_change,meanval,deviation,-3s,3s,-2s,2s,-1s,1s,impact,color,region,anomaly_points,load_date
0,2014-04-01 19:00:00,0.348,1.681749,-1.333749,-383.261322,0.528476,2.145213,-3.761950,4.818903,-3.225647,4.282599,-2.689344,3.746296,3,0,NEGATIVE,NaN,2014-06-01 00:00:00
1,2014-04-01 20:00:00,0.505,2.650134,-2.145134,-424.779086,0.704007,1.772303,-2.840599,4.248612,-2.397523,3.805537,-1.954448,3.362461,2,1,NEGATIVE,NaN,2014-06-01 01:00:00
2,2014-04-01 21:00:00,0.533,7.635795,-7.102795,-1332.606924,-0.093174,2.776140,-5.645453,5.459106,-4.951418,4.765071,-4.257383,4.071036,0,3,NEGATIVE,-7.102795,2014-06-01 02:00:00
3,2014-04-01 22:00:00,0.556,0.779133,-0.223133,-40.131861,-0.147666,2.771334,-5.690334,5.395002,-4.997501,4.702168,-4.304667,4.009335,3,0,NEGATIVE,NaN,2014-06-01 03:00:00
4,2014-04-01 23:00:00,0.511,0.514777,-0.003777,-0.739092,-0.214300,2.756158,-5.726615,5.298016,-5.037576,4.608976,-4.348536,3.919937,4,0,POSITIVE,NaN,2014-06-01 04:00:00
5,2014-04-02 00:00:00,0.228,0.193024,0.034976,15.340225,-0.218640,2.755681,-5.730002,5.292721,-5.041082,4.603801,-4.352162,3.914881,4,0,POSITIVE,NaN,2014-06-01 05:00:00
6,2014-04-02 01:00:00,0.179,0.183879,-0.004879,-2.725942,-0.324864,2.717391,-5.759647,5.109919,-5.080299,4.430571,-4.400951,3.751223,4,0,POSITIVE,NaN,2014-06-01 06:00:00
7,2014-04-02 02:00:00,0.216,0.146909,0.069091,31.986441,-0.711450,2.220917,-5.153284,3.730383,-4.598055,3.175154,-4.042825,2.619925,4,0,POSITIVE,NaN,2014-06-01 07:00:00
8,2014-04-02 03:00:00,0.172,0.197197,-0.025197,-14.649276,-0.867054,2.087130,-5.041314,3.307205,-4.519531,2.785423,-3.997749,2.263641,4,0,POSITIVE,NaN,2014-06-01 08:00:00
9,2014-04-02 04:00:00,0.214,0.161958,0.052042,24.318867,-0.872820,2.084245,-5.041310,3.295670,-4.520249,2.774609,-3.999187,2.253547,4,0,POSITIVE,NaN,2014-06-01 09:00:00


In [54]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from matplotlib import pyplot



def plot_anomaly(df,metric_name):
    #error = pd.DataFrame(Order_results.error.values)
    #df = df.sort_values(by='load_date', ascending=False)
    #df.load_date = pd.to_datetime(df['load_date'].astype(str), format="%Y%m%d")
    dates = df.date
    #meanval = error.rolling(window=window).mean()
    #deviation = error.rolling(window=window).std()
    #res = error
#upper_bond=meanval + (2 * deviation)
    #lower_bond=meanval - (2 * deviation)
#anomalies = pd.DataFrame(index=res.index, columns=res.columns)
    #anomalies[res < lower_bond] = res[res < lower_bond]
    #anomalies[res > upper_bond] = res[res > upper_bond]
    bool_array = (abs(df['anomaly_points']) > 0)
#And a subplot of the Actual Values.
    actuals = df["actuals"][-len(bool_array):]
    anomaly_points = bool_array * actuals
    anomaly_points[anomaly_points == 0] = np.nan
#Order_results['meanval']=meanval
    #Order_results['deviation']=deviation
#     color_map= {0: "palegreen", 1: "yellow", 2: "orange", 3: "red"}
#     table = go.Table(
#     domain=dict(x=[0, 1],
#                 y=[0, 0.3]),
#     columnwidth=[1, 2 ],
#     #columnorder=[0, 1, 2,],
#     header = dict(height = 20,
#                   values = [['<b>Date</b>'],['<b>Actual Values </b>'],
#                             ['<b>Predicted</b>'], ['<b>% Difference</b>'],['<b>Severity (0-3)</b>']],
#                  font = dict(color=['rgb(45, 45, 45)'] * 5, size=14),
#                   fill = dict(color='#d562be')),
#     cells = dict(values = [df.round(3)[k].tolist() for k in ['load_date', 'actuals', 'predicted',
#                                                                'percentage_change','color']],
#                  line = dict(color='#506784'),
#                  align = ['center'] * 5,
#                  font = dict(color=['rgb(40, 40, 40)'] * 5, size=12),
#                  #format = [None] + [",.4f"] + [',.4f'],
# #suffix=[None] * 4,
#                  suffix=[None] + [''] + [''] + ['%'] + [''],
#                  height = 27,
#                  #fill = dict(color=['rgb(235, 193, 238)', 'rgba(228, 222, 249, 0.65)']))
#                  fill=dict(color=  # ['rgb(245,245,245)',#unique color for the first column
#                       [df['color'].map(color_map)],
#                       )
#     ))
# df['ano'] = np.where(df['color']==3, df['error'], np.nan)
    anomalies = go.Scatter(name="Anomaly",
                       x=dates,
                       xaxis='x1',
                       yaxis='y1',
                       y=df['anomaly_points'],
                       mode='markers',
                       marker = dict(color ='red',
                      size = 11,line = dict(
                                         color = 'red',
                                         width = 2)))
    upper_bound = go.Scatter(hoverinfo="skip",
                         x=dates,
                         showlegend =False,
                         xaxis='x1',
                         yaxis='y1',
                         y=df['3s'],
                         marker=dict(color="#444"),
                         line=dict(
                             color=('rgb(23, 96, 167)'),
                             width=2,
                             dash='dash'),
                         fillcolor='rgba(68, 68, 68, 0.3)',
                         fill='tonexty')
    lower_bound = go.Scatter(name='Confidence Interval',
                          x=dates,
                         xaxis='x1',
                         yaxis='y1',
                          y=df['-3s'],
                          marker=dict(color="#444"),
                          line=dict(
                              color=('rgb(23, 96, 167)'),
                              width=2,
                              dash='dash'),
                          fillcolor='rgba(68, 68, 68, 0.3)',
                          fill='tonexty')
    
    Actuals = go.Scatter(name= 'Actuals',
                     x= dates,
                     y= df['actuals'],
                    xaxis='x2', yaxis='y2',
                     mode='lines',
                     marker=dict(size=12,
                                 line=dict(width=1),
                                 color='blue'))
    
    Predicted = go.Scatter(name= 'Predicted',
                     x= dates,
                     y= df['predicted'],
                    xaxis='x2', yaxis='y2',
                     mode='lines',
                     marker=dict(size=12,
                                 line=dict(width=1),
                                 color="orange"))
# create plot for error...
    Error = go.Scatter(name="Error",
                   x=dates, y=df['error'],
                   xaxis='x1',
                   yaxis='y1',
                   mode='lines',
                   marker=dict(size=12,
                               line=dict(width=1),
                               color="red"),
                   text="Error")
    anomalies_map = go.Scatter(name = "anomaly actual",
                                   showlegend=False,
                                   x=dates,
                                   y=anomaly_points,
                                   mode='markers',
                                   xaxis='x2',
                                   yaxis='y2',
                                    marker = dict(color ="red",
                                  size = 11,
                                 line = dict(
                                     color = "red",
                                     width = 2)))
    Mvingavrg = go.Scatter(name="Moving Average",
                           x=dates,
                           y=df['meanval'],
                           mode='lines',
                           xaxis='x1',
                           yaxis='y1',
                           marker=dict(size=12,
                                       line=dict(width=1),
                                       color="green"),
                           text="Moving average")
    axis=dict(
    showline=True,
    zeroline=False,
    showgrid=True,
    mirror=True,
    ticklen=4,
    tickfont=dict(size=18))
    
    layout = dict(
    width=1000,
    height=865,
    autosize=False,
    title= metric_name,
    margin = dict(t=75),
    showlegend=True,
    xaxis1=dict(axis, **dict(domain=[0, 1], anchor='y1', showticklabels=True)),
    xaxis2=dict(axis, **dict(domain=[0, 1], anchor='y2', showticklabels=True)),
    yaxis1=dict(axis, **dict(domain=[2 * 0.21 + 0.20 + 0.09, 1], anchor='x1', hoverformat='.2f')),
    yaxis2=dict(axis, **dict(domain=[0.21 + 0.12, 2 * 0.31 + 0.02], anchor='x2', hoverformat='.2f')))
    
    fig = go.Figure(data = [anomalies,anomalies_map,
                        upper_bound,lower_bound,Actuals,Predicted,
                        Mvingavrg,Error], layout = layout)
    fig.update_layout(template="plotly_white", xaxis=dict(showgrid=True),
        yaxis=dict(showgrid=True),font=dict(
        family="Arial",
        size=18
    ))
    fig.update_yaxes(title_text='other(kWh)')
    fig.update_layout(legend=dict(x=0.75, y=1.2))
    fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True)
    fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True)
    
#     fig.update_layout(showlegend=True, height=1000, width=800, title_text="Test set prediction", 
#                       template="plotly_white", font=dict(
#         family="Arial",
#         size=18
#     ))
    iplot(fig)




In [55]:
# {'air1': 0, 'furnace1': 1, 'dishwasher1': 2, 'regrigerator1': 3, 'other': 4}

result_df = generate_df_detect_anomalies(d, 10, 'other', x_test, x_test_localhour, x_predict)

classify_df = detect_classify_anomalies(result_df,12)
classify_df.reset_index(inplace=True)
del classify_df['index']
plot_anomaly(classify_df,"Abnormal Detection with Prediction")

## Statistical Analysis 

## Plotting the B matrices (basis functions)

In [ ]:
from matplotlib import cm

plt.figure(figsize=(16,12))

# row and column sharing
f, ((ax1, ax2, ax3)) = plt.subplots(3, 1, sharex='col', sharey='row', figsize=(16,12))
plt.rcParams.update({'font.size': 15})


B_list[0] = B_list[0]/np.sum(B_list[0],axis=1)[:,None]  # Normalize
ax1.pcolor(B_list[0], cmap = cm.Greys_r)
B_list[1] = B_list[1]/np.sum(B_list[1],axis=1)[:,None]  # Normalize
ax2.pcolor(B_list[1], cmap = cm.Greys_r)
B_list[2] = B_list[2]/np.sum(B_list[2],axis=1)[:,None]  # Normalize
ax3.pcolor(B_list[2], cmap = cm.Greys_r)

ax1.get_xaxis().set_visible(False)
ax1.get_yaxis().set_visible(False)
ax2.get_xaxis().set_visible(False)
ax2.get_yaxis().set_visible(False)
ax3.get_xaxis().set_visible(False)
ax3.get_yaxis().set_visible(False)

# plt.savefig(figure_directory+'basis')

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('classic')

# row and column sharing
f, ((ax1, ax2, ax3, ax4, ax5)) = plt.subplots(5, 1, sharex='col', sharey='row', figsize=(16,12))
plt.rcParams.update({'font.size': 15})


for base in range(7):
    ax1.plot(range(n),B_list[0][base,0:])
    ax2.plot(range(n),B_list[1][base,0:])
    ax3.plot(range(n),B_list[2][base,0:])
    ax4.plot(range(n),B_list[3][base,0:])
    ax5.plot(range(n),B_list[4][base,0:])

ax1.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom='off',      # ticks along the bottom edge are off
    top='off',         # ticks along the top edge are off
    labelbottom='off') # labels along the bottom edge are off
# ax1.set_ylim([0,0.04])
ax1.set_ylabel('Refrigerator')
ax2.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom='off',      # ticks along the bottom edge are off
    top='off',         # ticks along the top edge are off
    labelbottom='off') # labels along the bottom edge are off
# ax2.set_ylim([0,0.06])
ax2.set_ylabel('Dishwasher')
ax3.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom='off',      # ticks along the bottom edge are off
    top='off',         # ticks along the top edge are off
    labelbottom='off') # labels along the bottom edge are off
# ax3.set_ylim([0,0.02])
ax3.set_ylabel('Furnace')